In [32]:
# import the libraries
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import cv2 
from sklearn.model_selection import train_test_split

In [33]:
# Load the dataset
data = pd.read_csv('fer2013.csv')
data.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
# Preprocess the image 
def preprocess_image(image):
    image = np.array(image.split(), dtype='float32')
    image = image.reshape(48,48,1) # Reshape to 48x48 and add channel dimension
    image = image / 255.0 # Normalize to [0,1]
    return image

In [11]:
x = np.array([preprocess_image(img) for img in data['pixels']])
y = tf.keras.utils.to_categorical(data['emotion'], num_classes=7) # converts integer labels into one-hot encoded format.

In [10]:
# split the dataset into training and testing sets 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

# Model Building

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [13]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)), 
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)), 
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)), 
    Dropout(0.25),

    Flatten(),
    Dense(256,  activation='relu'),
    BatchNormalization(), 
    Dropout(0.25),
    Dense(7, activation='softmax')
])

C:\Users\ishit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# Model training 

In [15]:
history = model.fit(x_train,y_train, epochs=50, batch_size=64, validation_data=(x_test,y_test))

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 88s 180ms/step - accuracy: 0.2916 - loss: 2.0751 - val_accuracy: 0.3167 - val_loss: 1.7243
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 82s 182ms/step - accuracy: 0.4370 - loss: 1.4871 - val_accuracy: 0.3803 - val_loss: 1.5668
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - accuracy: 0.4866 - loss: 1.3510 - val_accuracy: 0.4819 - val_loss: 1.3562
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 415s 926ms/step - accuracy: 0.5120 - loss: 1.2732 - val_accuracy: 0.4968 - val_loss: 1.3631
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - accuracy: 0.5414 - loss: 1.2160 - val_accuracy: 0.5169 - val_loss: 1.3058
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 81s 179ms/step - accuracy: 0.5598 - loss: 1.1614 - val_accuracy: 0.5188 - val_loss: 1.2803
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 83s 181ms/step - accuracy: 0.5765 - loss: 1.1208 - val_accuracy: 0.5539 - val_loss: 1.1880
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 82s 181ms/step - accuracy: 0.5948 - loss: 

In [20]:
# Evaluate the model performance
train_accuracy = model.history.history['accuracy']
test_accuracy = model.history.history['val_accuracy']

# Inference

In [26]:
from PIL import Image

In [41]:
def predict_emotion(image):
    image = preprocess_image(image)
    image = np.expand_dims(image, axis=0) # Add batch dimension
    prediction = model.predict(image)
    emotion = np.argmax(prediction)
    return emotion

In [48]:
# Example usage 
new_image = "C:\\Users\\ishit\\OneDrive\\Attachments\\pandas\\WhatsApp Image .jpg"
emotion = predict_emotion(new_image)
print(f'Predicted emotion: {emotion}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted emotion: 2


In [47]:
new_image = "C:\\Users\\ishit\\OneDrive\\Attachments\\pandas\\pexels-justin.jpg"
emotion = predict_emotion(new_image)
print(f'Predicted emotion: {emotion}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted emotion: 5


In [ ]:
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).